In [ ]:
!nvidia-smi

In [ ]:
%%time
!pip install openmim
!mim install mmdet

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import sklearn
import torchvision
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import PIL
import json
from PIL import Image, ImageEnhance
import albumentations as A
import mmdet
import mmcv
from albumentations.pytorch import ToTensorV2
import seaborn as sns
import glob
from pathlib import Path
import pycocotools
from pycocotools import mask
import numpy.random
import random
import cv2
import re
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import inference_detector, init_detector, show_result_pyplot, set_random_seed
from mmcv import Config

In [ ]:
!mkdir mmdetection/checkpoint
!wget https://download.openmmlab.com/mmdetection/v2.0/swin/mask_rcnn_swin-s-p4-w7_fpn_fp16_ms-crop-3x_coco/mask_rcnn_swin-s-p4-w7_fpn_fp16_ms-crop-3x_coco_20210903_104808-b92c91f1.pth -P checkpoint/

In [ ]:
%%writefile labels.txt
shsy5y
cort
astro

In [ ]:
cfg = Config.fromfile('mmdetection/configs/swin/mask_rcnn_swin-s-p4-w7_fpn_fp16_ms-crop-3x_coco.py')
print(cfg.pretty_text)

In [ ]:
fold=2

In [ ]:
cfg.dataset_type = 'CustomDataset'
cfg.classes = 'labels.txt'

cfg.model.roi_head.bbox_head.num_classes=3

cfg.model.roi_head.mask_head.num_classes=3

cfg.data.test.type = 'CustomDataset'
cfg.data.test.classes = 'labels.txt'
cfg.data.test.ann_file = f'/kaggle/input/scs-10-folds-mmdet/tenFoldsMMdet/tenFolds_MMdet/mmdet_val_fold2.pkl'
cfg.data.test.img_prefix = '/kaggle/input/scs-10-folds-mmdet/SCS_images/SCS_images/'

cfg.data.train.type = 'CustomDataset'
cfg.data.train.ann_file = f'/kaggle/input/scs-10-folds-mmdet/tenFoldsMMdet/tenFolds_MMdet/mmdet_trn_fold2.pkl'
cfg.data.train.img_prefix = '/kaggle/input/scs-10-folds-mmdet/SCS_images/SCS_images/'
cfg.data.train.classes = 'labels.txt'

cfg.data.val.type = 'CustomDataset'
cfg.data.val.ann_file = f'/kaggle/input/scs-10-folds-mmdet/tenFoldsMMdet/tenFolds_MMdet/mmdet_val_fold2.pkl'
cfg.data.val.img_prefix = '/kaggle/input/scs-10-folds-mmdet/SCS_images/SCS_images/'
cfg.data.val.classes = 'labels.txt'


# cfg.train_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(
#         type='LoadAnnotations', with_bbox=True, with_mask=True,
#         ),
#     dict(
#         type='Resize',
#         img_scale=[(1333, 1333), (1280, 1280), (1024, 1024)],
#         multiscale_mode='value',
#         keep_ratio=True),
#     dict(
#         type='RandomFlip',
#         direction=['horizontal', 'vertical'],
#         flip_ratio=0.5),
#     dict(
#         type='Normalize',
#         mean=[123.68, 116.779, 103.939],
#         std=[58.393, 57.12, 57.375],
#         to_rgb=True),
#     dict(type='Pad', size_divisor=32),
#     dict(type='DefaultFormatBundle'),
#     dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks'])
# ]

# cfg.val_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(
#         type='MultiScaleFlipAug',
#         img_scale=(1280, 1280),
#         flip=True,
#         transforms=[
#             dict(type='Resize', keep_ratio=True),
#             dict(type='RandomFlip', direction=['horizontal', 'vertical']),
#             dict(
#                 type='Normalize',
#                 mean=[123.68, 116.779, 103.939],
#                 std=[58.393, 57.12, 57.375],
#                 to_rgb=True),
#             dict(type='Pad', size_divisor=32),
#             dict(type='ImageToTensor', keys=['img']),
#             dict(type='Collect', keys=['img'])
#         ])
# ]


# cfg.test_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(
#         type='MultiScaleFlipAug',
#         img_scale=(1280, 1280),
#         flip=True,
#         transforms=[
#             dict(type='Resize', keep_ratio=True),
#             dict(type='RandomFlip', direction=['horizontal', 'vertical']),
#             dict(
#                 type='Normalize',
#                 mean=[123.68, 116.779, 103.939],
#                 std=[58.393, 57.12, 57.375],
#                 to_rgb=True),
#             dict(type='Pad', size_divisor=32),
#             dict(type='ImageToTensor', keys=['img']),
#             dict(type='Collect', keys=['img'])
#         ])
# ]

# cfg.data.train.pipeline = cfg.train_pipeline
# cfg.data.val.pipeline = cfg.val_pipeline
# cfg.data.test.pipeline = cfg.test_pipeline


cfg.model.test_cfg.rcnn.max_per_img = 200
cfg.load_from="checkpoint/mask_rcnn_swin-s-p4-w7_fpn_fp16_ms-crop-3x_coco_20210903_104808-b92c91f1.pth"
cfg.work_dir = 'model_output'

cfg.runner = dict(type='IterBasedRunner', max_iters=1000)
cfg.optimizer=dict(type='Adam', lr=0.0005, weight_decay=0.0001)
cfg.lr_config = dict(
    policy='CosineAnnealing', 
    by_epoch=False,
    warmup='linear', 
    warmup_iters=250,
    warmup_ratio=0.001,
    min_lr=1e-07)

cfg.data.samples_per_gpu = 1
cfg.data.workers_per_gpu = 1

cfg.evaluation.interval=20005
cfg.evaluation.metric='loss'

cfg.checkpoint_config.interval = 300
cfg.runner.max_iters = 40000
cfg.log_config.interval = 50

cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.fp16 = dict(loss_scale=512.0)
meta = dict()
meta['config'] = cfg.pretty_text

print(f'Config:\n{cfg.pretty_text}')

In [ ]:
datasets = [build_dataset(cfg.data.train)]
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES

mmcv.mkdir_or_exist(os.path.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True, meta=meta)

In [ ]:
 !mv /content/model_output/iter_18900.pth /content/drive/MyDrive/MMD_CRCNN1.pth